# Inference time measurement 

To be able to run this experimentation, [Grid2op](https://github.com/rte-france/Grid2Op) (1.7.1) and [LightSim2Grid](https://github.com/BDonnot/lightsim2grid) (0.7.0) are required. See the corresponding links for more information concerning their installation.

In [1]:
import pathlib
import numpy as np
from pprint import pprint
from matplotlib import pyplot as plt
from lips.benchmark.powergridBenchmark import PowerGridBenchmark
from lips.utils import get_path

# TOC
- [Device Selection](#device)
- [Environment: `l2rpn_case14_sandbox`](#env_case14)
    - [Benchmark Selection](#benchmark_select)
      - [Benchmark1](#benchmark1)
      - [Benchmark2](#benchmark2)
- [Environment: `l2rpn_neurips_2020_track1_small`](#env_nips)

## Select a device on which the model should be evaluated <a id="device"></a>
On the basis of GPU or CPU device, the results may be vary significantly. You can choose one of the following block in each kernel to using whether GPU or CPU device. 

### Using A GPU

In [2]:
import tensorflow as tf
device = "GPU"
gpus = tf.config.experimental.list_physical_devices(device)
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[3], device)
    except RuntimeError as e:
        # Visible devices must be set at program startup
        print(e)

### Using CPU

In [3]:
import os
device = "CPU"
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf

### Show the list of used devices

In [3]:
tf.config.experimental.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

# Environment : "l2rpn_case14_sandbox" <a id="env_case14"></a>

In [2]:
# indicate required paths
LIPS_PATH = pathlib.Path().resolve().parent # it is supposed that the notebook had run from getting_started folder
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_case14_sandbox"
BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_case14_sandbox.ini"
SIM_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "simulators"
BASELINES_PATH = LIPS_PATH / "trained_baselines" / "powergrid"
TRAINED_MODEL_PATH = LIPS_PATH / "trained_models" / "powergrid"
EVALUATION_PATH = LIPS_PATH / "evaluation_results" / "PowerGrid"
LOG_PATH = LIPS_PATH / "lips_logs.log"

## Select a benchmark <a id="benchmark_select"></a>

### Benchmark1 <a id="benchmark1"></a>

In [5]:
benchmark1 = PowerGridBenchmark(benchmark_name="Benchmark1",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

### Physics solver time 

Physics solver takes 0.02 miliseconds to solve a powerflow. However the physics solver time for the benchmark1 is computed as follows:
$$\underbrace{3}_\textrm{# situations} \times \underbrace{6}_\textrm{# topologies} \times \frac{\textrm{security analysis time}}{20}$$, where $20$ is number of the lines in this benchmark. In our case the *security analysis* takes 0.02 ms to be solved. So the solver time is $3 \times 6 \times 0.02 = 0.36 ms$

### Select a model 

#### Fully connected model

In [8]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark1",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

##### Load it from a saved model

In [9]:
LOAD_PATH = get_path(BASELINES_PATH, benchmark1)
tf_fc.restore(LOAD_PATH)

2022-06-06 20:13:02.772778: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


##### Evaluate it multiple times and return the inference times 

In [ ]:
# evaluate the inference time
time_ml = list()
time_ind = list()
for i in range(10):
    tf_fc_metrics = benchmark1.evaluate_simulator(augmented_simulator=tf_fc,
                                                  eval_batch_size=128,
                                                  dataset="all",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )
    time_ml.append(tf_fc_metrics["test"]["ML"]["time_inf"])
    time_ind.append(tf_fc_metrics["test"]["IndRed"]["time_inf"])

The inference time should be reported per powerflow. So it should be devided by $10000$ which represents the size of the data in test data dataset. For the sake of comparison which physics solver, we would report it in miliseconds. So, it should be multiplied by $1000$. 

In [32]:
inf_time_IndRed = (np.mean(time_ind) / 10000) * 1000
inf_time_ML = (np.mean(time_ml) / 10000) * 1000
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

Inference time from Industrial Readiness category and using GPU: 0.018338852387387305
Inference time from Machine Learning category and using GPU: 0.0044172982173040515


In [11]:
inf_time_IndRed = (np.mean(time_ind) / 10000) * 1000
inf_time_ML = (np.mean(time_ml) / 10000) * 1000
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

Inference time from Industrial Readiness category and using CPU: 0.02155673010274768
Inference time from Machine Learning category and using CPU: 0.004848378957249224


##### Computing the speed up 

In [33]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (0.36 / inf_time_IndRed)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (0.36 / inf_time_ML)))

Speed up factor from IndRed point of view using FullyConnected model and GPU is: 19.63
Speed up factor from ML point of view using FullyConnected model and GPU is: 81.50


In [12]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (0.36 / inf_time_IndRed)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (0.36 / inf_time_ML)))

Speed up factor from IndRed point of view using FullyConnected model and CPU is: 16.70
Speed up factor from ML point of view using FullyConnected model and CPU is: 74.25


#### LeapNet model 

In [13]:
from lips.augmented_simulators.tensorflow_models import LeapNet
from lips.dataset.scaler import PowerGridScaler

leap_net = LeapNet(name="tf_leapnet",
                   bench_config_path=BENCH_CONFIG_PATH,
                   bench_config_name="Benchmark1",
                   sim_config_path=SIM_CONFIG_PATH / "tf_leapnet.ini",
                   sim_config_name="DEFAULT",
                   scaler=PowerGridScaler,
                   log_path=LOG_PATH)

Load the model for the convenience.

In [14]:
LOAD_PATH = get_path(BASELINES_PATH, benchmark1)
leap_net.restore(path=LOAD_PATH)

##### Evaluate it multiple times

In [ ]:
# evaluate the inference time
time_ml = list()
time_ind = list()
for i in range(10):
    leapnet_metrics = benchmark1.evaluate_simulator(augmented_simulator=leap_net,
                                                    eval_batch_size=128,
                                                    dataset="all",
                                                    shuffle=False,
                                                    save_path=None,
                                                    save_predictions=False
                                                 )
    time_ml.append(leapnet_metrics["test"]["ML"]["time_inf"])
    time_ind.append(leapnet_metrics["test"]["IndRed"]["time_inf"])

In [19]:
inf_time_IndRed = (np.mean(time_ind) / 10000) * 1000
inf_time_ML = (np.mean(time_ml) / 10000) * 1000
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

Inference time from Industrial Readiness category and using GPU: 0.14272481479216367
Inference time from Machine Learning category and using GPU: 0.10670942533994095


In [16]:
inf_time_IndRed = (np.mean(time_ind) / 10000) * 1000
inf_time_ML = (np.mean(time_ml) / 10000) * 1000
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

Inference time from Industrial Readiness category and using CPU: 0.1397597827226855
Inference time from Machine Learning category and using CPU: 0.11120240544900296


##### Computing the speed up

In [25]:
print("Speed up factor from IndRed point of view using LeapNet model and {} is: {:.2f}".format(device, (0.36 / inf_time_IndRed)))
print("Speed up factor from ML point of view using LeapNet model and {} is: {:.2f}".format(device, (0.36 / inf_time_ML)))

Speed up factor from IndRed point of view using LeapNet model and GPU is: 2.52
Speed up factor from ML point of view using LeapNet model and GPU is: 3.37


In [17]:
print("Speed up factor from IndRed point of view using LeapNet model and {} is: {:.2f}".format(device, (0.36 / inf_time_IndRed)))
print("Speed up factor from ML point of view using LeapNet model and {} is: {:.2f}".format(device, (0.36 / inf_time_ML)))

Speed up factor from IndRed point of view using LeapNet model and CPU is: 2.58
Speed up factor from ML point of view using LeapNet model and CPU is: 3.24


## Benchmark2 <a id="benchmark2"></a>

In [5]:
benchmark2 = PowerGridBenchmark(benchmark_name="Benchmark2",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

### Physics solver time 

Apart of first benchmark (security analysis) where the inverse of `Y_bus` matrix could be computed once and the observatiosn could be generated in parallel, the other two benchmarks require to compute this matrix inversion for each flow computation. Hence, to be able to compute the physics solver time, we can use the following times:

$$ \textrm{_timer_preproc} + \textrm{_timer_solver} + \textrm{_timer_postproc}$$

The details explanations for each of these steps are provided [here](https://docs.google.com/document/d/1MWj1NUj4hbPuMMzu0ty9Md_qk9mnLz88chEOi3TwJJU/edit#).

In the next cell we compute the flow over 1000 observations, and finally we report the time required for one powerflow to be comptued.

In [ ]:
benchmark2_tmp = PowerGridBenchmark(benchmark_name="Benchmark2",
                                    benchmark_path=None,
                                    load_data_set=False,
                                    log_path=LOG_PATH,
                                    config_path=BENCH_CONFIG_PATH
                                   )
time_physics_solver = list()
for i in range(10):
    benchmark2_tmp.generate(nb_sample_train=int(1),
                            nb_sample_val=int(1),
                            nb_sample_test=int(10000),
                            nb_sample_test_ood_topo=int(1),
                           )
    total_time = benchmark2_tmp.test_simulator._simulator.backend._timer_postproc + \
                 benchmark2_tmp.test_simulator._simulator.backend._timer_preproc + \
                 benchmark2_tmp.test_simulator._simulator.backend._timer_solver
    time_physics_solver.append(total_time)

In [61]:
physics_solver_time = np.mean(time_physics_solver)
print("Physics solver computation time for benchmark 2 and one power flow is: {} s".format(physics_solver_time))

Physics solver computation time for benchmark 2 and one power flow is: 0.04083034829236567 s


In [6]:
physics_solver_time = 0.04083034829236567

### Select a model

#### Fully Connected model

In [7]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark1",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

In [8]:
LOAD_PATH = get_path(BASELINES_PATH, benchmark2)
tf_fc.restore(LOAD_PATH)

2022-06-07 11:51:24.243887: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-07 11:51:24.690001: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46720 MB memory:  -> device: 3, name: NVIDIA RTX A6000, pci bus id: 0000:c3:00.0, compute capability: 8.6


##### Evaluate it multiple times and return the inference times 

In [ ]:
# evaluate the inference time
time_ml = list()
time_ind = list()
for i in range(10):
    tf_fc_metrics = benchmark2.evaluate_simulator(augmented_simulator=tf_fc,
                                                  eval_batch_size=128,
                                                  dataset="all",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )
    time_ml.append(tf_fc_metrics["test"]["ML"]["time_inf"])
    time_ind.append(tf_fc_metrics["test"]["IndRed"]["time_inf"])

In [70]:
inf_time_IndRed = np.mean(time_ind)
inf_time_ML = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

Inference time from Industrial Readiness category and using CPU: 0.21410137366037815
Inference time from Machine Learning category and using CPU: 0.05234153128694743


In [10]:
inf_time_IndRed = np.mean(time_ind)
inf_time_ML = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

Inference time from Industrial Readiness category and using GPU: 0.017661474130582066
Inference time from Machine Learning category and using GPU: 0.004774301669094712


##### Computing the speed up 

In [71]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML)))

Speed up factor from IndRed point of view using FullyConnected model and CPU is: 0.19
Speed up factor from ML point of view using FullyConnected model and CPU is: 0.78


In [11]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML)))

Speed up factor from IndRed point of view using FullyConnected model and GPU is: 2.31
Speed up factor from ML point of view using FullyConnected model and GPU is: 8.55


#### LeapNet model 

In [12]:
from lips.augmented_simulators.tensorflow_models import LeapNet
from lips.dataset.scaler import PowerGridScaler

leap_net = LeapNet(name="tf_leapnet",
                   bench_config_path=BENCH_CONFIG_PATH,
                   bench_config_name="Benchmark1",
                   sim_config_path=SIM_CONFIG_PATH / "tf_leapnet.ini",
                   sim_config_name="DEFAULT",
                   scaler=PowerGridScaler,
                   log_path=LOG_PATH)

Load the model for the convenience.

In [13]:
LOAD_PATH = get_path(BASELINES_PATH, benchmark2)
leap_net.restore(path=LOAD_PATH)

##### Evaluate it multiple times

In [ ]:
# evaluate the inference time
time_ml = list()
time_ind = list()
for i in range(10):
    leapnet_metrics = benchmark2.evaluate_simulator(augmented_simulator=leap_net,
                                                    eval_batch_size=128,
                                                    dataset="all",
                                                    shuffle=False,
                                                    save_path=None,
                                                    save_predictions=False
                                                 )
    time_ml.append(leapnet_metrics["test"]["ML"]["time_inf"])
    time_ind.append(leapnet_metrics["test"]["IndRed"]["time_inf"])

In [65]:
inf_time_IndRed = np.mean(time_ind)
inf_time_ML = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

Inference time from Industrial Readiness category and using CPU: 0.22096358130220323
Inference time from Machine Learning category and using CPU: 0.05369499467778951


In [15]:
inf_time_IndRed = np.mean(time_ind)
inf_time_ML = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

Inference time from Industrial Readiness category and using GPU: 0.1634221963491291
Inference time from Machine Learning category and using GPU: 0.1115585584449582


##### Computing speed up

In [66]:
print("Speed up factor from IndRed point of view using LeapNet model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed)))
print("Speed up factor from ML point of view using LeapNet model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML)))

Speed up factor from IndRed point of view using LeapNet model and CPU is: 0.18
Speed up factor from ML point of view using LeapNet model and CPU is: 0.76


In [16]:
print("Speed up factor from IndRed point of view using LeapNet model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed)))
print("Speed up factor from ML point of view using LeapNet model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML)))

Speed up factor from IndRed point of view using LeapNet model and GPU is: 0.25
Speed up factor from ML point of view using LeapNet model and GPU is: 0.37


# Environment : "l2rpn_neurips_2020_track1_small" <a id="env_nips"></a>

In [4]:
# indicate required paths
LIPS_PATH = pathlib.Path().resolve().parent # it is supposed that the notebook had run from getting_started folder
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_neurips_2020_track1_small"
BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_neurips_2020_track1_small.ini"
SIM_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "simulators"
BASELINES_PATH = LIPS_PATH / "trained_baselines" / "powergrid"
TRAINED_MODEL_PATH = LIPS_PATH / "trained_models" / "powergrid"
EVALUATION_PATH = LIPS_PATH / "evaluation_results" / "PowerGrid"
LOG_PATH = LIPS_PATH / "lips_logs.log"

## Benchmark2 <a id="benchmark2"></a>

In [5]:
benchmark2 = PowerGridBenchmark(benchmark_name="Benchmark2",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

### Physics solver time 

Apart of first benchmark (security analysis) where the inverse of `Y_bus` matrix could be computed once and the observatiosn could be generated in parallel, the other two benchmarks require to compute this matrix inversion for each flow computation. Hence, to be able to compute the physics solver time, we can use the following times:

$$ \textrm{_timer_preproc} + \textrm{_timer_solver} + \textrm{_timer_postproc}$$

The details explanations for each of these steps are provided [here](https://docs.google.com/document/d/1MWj1NUj4hbPuMMzu0ty9Md_qk9mnLz88chEOi3TwJJU/edit#).

In the next cell we compute the flow over 1000 observations, and finally we report the time required for one powerflow to be comptued.

In [ ]:
benchmark2_tmp = PowerGridBenchmark(benchmark_name="Benchmark2",
                                    benchmark_path=None,
                                    load_data_set=False,
                                    log_path=LOG_PATH,
                                    config_path=BENCH_CONFIG_PATH
                                   )
time_physics_solver = list()
for i in range(10):
    benchmark2_tmp.generate(nb_sample_train=int(1),
                            nb_sample_val=int(1),
                            nb_sample_test=int(10000),
                            nb_sample_test_ood_topo=int(1),
                           )
    total_time = benchmark2_tmp.test_simulator._simulator.backend._timer_postproc + \
                 benchmark2_tmp.test_simulator._simulator.backend._timer_preproc + \
                 benchmark2_tmp.test_simulator._simulator.backend._timer_solver
    time_physics_solver.append(total_time)

In [9]:
physics_solver_time = np.mean(time_physics_solver)
print("Physics solver computation time for benchmark 2 and one power flow is: {} s".format(physics_solver_time))

Physics solver computation time for benchmark 2 and one power flow is: 0.05780877992510795 s


In [6]:
physics_solver_time = 0.05780877992510795

### Select a model

#### Fully Connected model

In [7]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark1",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

In [8]:
LOAD_PATH = get_path(TRAINED_MODEL_PATH, benchmark2)
tf_fc.restore(LOAD_PATH)

2022-06-07 12:54:23.513096: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-07 12:54:23.968420: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 516 MB memory:  -> device: 3, name: NVIDIA RTX A6000, pci bus id: 0000:c3:00.0, compute capability: 8.6


##### Evaluate it multiple times and return the inference times 

In [ ]:
# evaluate the inference time
time_ml = list()
time_ind = list()
for i in range(10):
    tf_fc_metrics = benchmark2.evaluate_simulator(augmented_simulator=tf_fc,
                                                  eval_batch_size=128,
                                                  dataset="all",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )
    time_ml.append(tf_fc_metrics["test"]["ML"]["time_inf"])
    time_ind.append(tf_fc_metrics["test"]["IndRed"]["time_inf"])

In [70]:
inf_time_IndRed = np.mean(time_ind)
inf_time_ML = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

Inference time from Industrial Readiness category and using CPU: 0.21410137366037815
Inference time from Machine Learning category and using CPU: 0.05234153128694743


In [11]:
inf_time_IndRed = np.mean(time_ind)
inf_time_ML = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

Inference time from Industrial Readiness category and using GPU: 0.20453928930219262
Inference time from Machine Learning category and using GPU: 0.0727451688842848


##### Computing the speed up 

In [71]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML)))

Speed up factor from IndRed point of view using FullyConnected model and CPU is: 0.19
Speed up factor from ML point of view using FullyConnected model and CPU is: 0.78


In [12]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML)))

Speed up factor from IndRed point of view using FullyConnected model and GPU is: 0.28
Speed up factor from ML point of view using FullyConnected model and GPU is: 0.79


<h1 style="color:red">Test</h1>

In [13]:
import grid2op
from lightsim2grid import LightSimBackend
env = grid2op.make("l2rpn_case14_sandbox", backend=LightSimBackend())

In [14]:
from grid2op.Agent import DoNothingAgent, RandomAgent

In [15]:
agent = DoNothingAgent(env.action_space)

In [16]:
obs = env.reset()

In [17]:
reward = env.reward_range[0]
done = False
action = agent.act(obs, reward)

In [29]:
print("preprocess time : ", env.backend._timer_preproc)
print("solver time : ", env.backend._timer_solver)
print("postprocess time : ", env.backend._timer_postproc)

preprocess time :  6.270804442465305e-05
solver time :  8.620298467576504e-05
postprocess time :  0.00012840190902352333


In [19]:
done=True
while done:
    obs, reward, info, done = env.step(action)

Grid2OpException: Grid2OpException "Impossible to make a step with a non initialized backend. Have you called "env.reset()" after the last game over ?"

In [20]:
print("preprocess time : ", env.backend._timer_preproc)
print("solver time : ", env.backend._timer_solver)
print("postprocess time : ", env.backend._timer_postproc)

preprocess time :  0.016066802898421884
solver time :  0.0222038880456239
postprocess time :  0.079864083789289


In [33]:
print("preprocess time : ", env.backend._timer_preproc)
print("solver time : ", env.backend._timer_solver)
print("postprocess time : ", env.backend._timer_postproc)

preprocess time :  0.00024596485309302807
solver time :  0.0002572669181972742
postprocess time :  0.0014893850311636925


In [35]:
print("preprocess time : ", env.backend._timer_preproc)
print("solver time : ", env.backend._timer_solver)
print("postprocess time : ", env.backend._timer_postproc)

preprocess time :  0.00032253097742795944
solver time :  0.0003424778115004301
postprocess time :  0.002225663047283888


In [38]:
benchmark2_tmp = PowerGridBenchmark(benchmark_name="Benchmark2",
                                    benchmark_path=None,
                                    load_data_set=False,
                                    log_path=LOG_PATH,
                                    config_path=BENCH_CONFIG_PATH
                                   )

In [47]:
benchmark2_tmp.generate(nb_sample_train=int(1),
                        nb_sample_val=int(1),
                        nb_sample_test=int(10000),
                        nb_sample_test_ood_topo=int(1),
                       )

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.24it/s]


In [58]:
simulator = benchmark2_tmp.test_simulator._simulator
print("preprocess time : ", simulator.backend._timer_preproc)
print("solver time : ", simulator.backend._timer_solver)
print("postprocess time : ", simulator.backend._timer_postproc)

preprocess time :  0.007299850229173899
solver time :  0.012052051490172744
postprocess time :  0.021055438788607717


In [59]:
solver_time = (simulator.backend._timer_preproc + simulator.backend._timer_solver + simulator.backend._timer_postproc)
print("IndRed time: ", solver_time / 2e-2)
print("ML time: ", solver_time / 5e-3)

IndRed time:  2.020367025397718
ML time:  8.081468101590872


In [57]:
solver_time

0.0006371969357132912

In [49]:
simulator = benchmark2_tmp.training_simulator._simulator
print("preprocess time : ", simulator.backend._timer_preproc)
print("solver time : ", simulator.backend._timer_solver)
print("postprocess time : ", simulator.backend._timer_postproc)

preprocess time :  0.00016485294327139854
solver time :  0.00018332712352275848
postprocess time :  0.000358447665348649


In [50]:
simulator = benchmark2_tmp.val_simulator._simulator
print("preprocess time : ", simulator.backend._timer_preproc)
print("solver time : ", simulator.backend._timer_solver)
print("postprocess time : ", simulator.backend._timer_postproc)

preprocess time :  0.00012895278632640839
solver time :  0.000169272068887949
postprocess time :  0.0003389720804989338
